Cemil can öz

2312729013


github repo bağlantısı:https://github.com/Cemilcanoz/python-kodlari-ile-kar-nca-kolonisi-algoritmas-Bursa-Belediyisnde-geri-d-n-m-arac-optimizasyon


Senaryo3: Bursa Belediyesine bağlı geri dönüşüm aracı, haftalık olarak 12 farklı liseden atık toplamaktadır. Yakıt ve zaman kazancı için en verimli rotayı belirleyiniz.

In [ ]:
!pip install googlemaps streamlit streamlit-folium folium pyngrok

In [ ]:
import os
import shutil

# --- 1. PROJE KLASÖRÜNÜ  OLUŞTUR ---
base_dir = "/content/aco_ilac_rutasi"

if os.path.exists(base_dir):
    shutil.rmtree(base_dir)

# Gerekli alt klasörler
subfolders = ["data", "core", "visual", ".streamlit", "figure"]
for folder in subfolders:
    os.makedirs(os.path.join(base_dir, folder), exist_ok=True)

# ---------------------------------------------------------
# 1. DOSYA: .gitignore (GÜVENLİK)
# ---------------------------------------------------------
with open(f"{base_dir}/.gitignore", "w", encoding="utf-8") as f:
    f.write(".env\n.streamlit/secrets.toml\n__pycache__/\n*.pkl\n.DS_Store")

# ---------------------------------------------------------
# 2. DOSYA: data/coordinates.py (BURSA OKULLARI)
# ---------------------------------------------------------
with open(f"{base_dir}/data/coordinates.py", "w", encoding="utf-8") as f:
    f.write('''
SCHOOLS = [
    "Osmangazi Belediyesi, Bursa",  # MERKEZ (Çıkış Noktası)
    "Tofaş Fen Lisesi, Bursa",
    "Bursa Anadolu Lisesi, Bursa",
    "Ulubatlı Hasan Anadolu Lisesi, Bursa",
    "Ahmet Erdem Anadolu Lisesi, Bursa",
    "Çınar Anadolu Lisesi, Bursa",
    "Osmangazi Gazi Anadolu Lisesi, Bursa",
    "Nilüfer İMKB Fen Lisesi, Bursa",
    "Bursa Erkek Lisesi, Bursa",
    "Hürriyet Anadolu Lisesi, Bursa",
    "Cumhuriyet Anadolu Lisesi, Bursa",
    "Barış Anadolu Lisesi, Bursa",
    "Ali Karasu Anadolu Lisesi, Bursa"
]
''')

# ---------------------------------------------------------
# 3. DOSYA: core/matrix_utils.py (GOOGLE MAPS MESAFE)
# ---------------------------------------------------------
with open(f"{base_dir}/core/matrix_utils.py", "w", encoding="utf-8") as f:
    f.write('''
import googlemaps
import numpy as np
import time
import streamlit as st

def get_coordinates(location_name, gmaps_client):
    try:
        res = gmaps_client.geocode(location_name)
        if res:
            loc = res[0]['geometry']['location']
            return loc['lat'], loc['lng']
    except: return None, None
    return None, None

def create_distance_matrix(locations, gmaps_client):
    size = len(locations)
    matrix = np.zeros((size, size))

    prog = st.progress(0)
    status = st.empty()

    for i in range(size):
        status.text(f"Mesafe verisi alınıyor: {i+1}/{size}")
        prog.progress((i + 1) / size)
        try:
            res = gmaps_client.distance_matrix([locations[i]], locations, mode="driving")
            if res['status'] == 'OK':
                elements = res['rows'][0]['elements']
                for j in range(size):
                    if i == j: matrix[i][j] = 0
                    else:
                        if elements[j]['status'] == 'OK':
                            matrix[i][j] = elements[j]['distance']['value']
                        else: matrix[i][j] = 999999
        except: pass
        time.sleep(0.1)

    prog.empty()
    status.empty()
    return matrix
''')

# ---------------------------------------------------------
# 4. DOSYA: core/ant_algorithm.py (ACO MANTIĞI)
# ---------------------------------------------------------
with open(f"{base_dir}/core/ant_algorithm.py", "w", encoding="utf-8") as f:
    f.write('''
import numpy as np

class AntColonyOptimization:
    def __init__(self, distances, n_ants, n_best, n_iterations, decay, alpha=1, beta=1):
        self.distances = distances
        self.pheromone = np.ones(self.distances.shape) / len(distances)
        self.n_ants, self.n_best, self.n_iterations = n_ants, n_best, n_iterations
        self.decay, self.alpha, self.beta = decay, alpha, beta
        self.best_path, self.best_dist, self.history = None, float('inf'), []

    def run(self):
        for _ in range(self.n_iterations):
            paths = []
            for _ in range(self.n_ants):
                path = self._gen_path(0)
                dist = self._dist(path)
                paths.append((path, dist))

            self._update(paths)
            self.history.append(self.best_dist)
        return self.best_path, self.best_dist, self.history

    def _gen_path(self, start):
        path, visited = [start], {start}
        for _ in range(len(self.distances) - 1):
            move = self._pick(self.pheromone[path[-1]], self.distances[path[-1]], visited)
            path.append(move)
            visited.add(move)
        path.append(start)
        return path

    def _pick(self, pher, dist, visited):
        with np.errstate(divide='ignore', invalid='ignore'):
            row = (pher ** self.alpha) * ((1.0 / (dist + 1e-6)) ** self.beta)
        row[list(visited)] = 0
        if row.sum() == 0:
            return np.random.choice(list(set(range(len(self.distances))) - visited))
        return np.random.choice(range(len(self.distances)), p=row/row.sum())

    def _dist(self, path):
        return sum(self.distances[path[i]][path[i+1]] for i in range(len(path)-1))

    def _update(self, paths):
        sorted_p = sorted(paths, key=lambda x: x[1])
        if sorted_p[0][1] < self.best_dist:
            self.best_dist, self.best_path = sorted_p[0][1], sorted_p[0][0]

        self.pheromone *= self.decay
        for p, d in sorted_p[:self.n_best]:
            for i in range(len(p)-1):
                self.pheromone[p[i]][p[i+1]] += 1.0 / (d + 1e-6)
''')

# ---------------------------------------------------------
# 5. DOSYA: main.py (PANDAS VE GÜVENLİK EKLENTİLİ)
# ---------------------------------------------------------
with open(f"{base_dir}/main.py", "w", encoding="utf-8") as f:
    f.write('''
import streamlit as st
import googlemaps
import folium
from streamlit_folium import st_folium
import numpy as np
import pickle
import os
import time
import pandas as pd

from data.coordinates import SCHOOLS
from core.matrix_utils import get_coordinates, create_distance_matrix
from core.ant_algorithm import AntColonyOptimization

st.set_page_config(layout="wide", page_title="Bursa ACO")
st.title("🚛 Bursa Geri Dönüşüm Rota Optimizasyonu")

SAVE_FILE = "sonuclar.pkl"

st.sidebar.header("⚙️ Ayarlar")

# API Anahtarı Yönetimi
default_api_key = ""
try:
    if "google_maps_api_key" in st.secrets:
        default_api_key = st.secrets["google_maps_api_key"]
except: pass

api_key = st.sidebar.text_input("Google Maps API Key", value=default_api_key, type="password")

n_ants = st.sidebar.slider("Karınca Sayısı", 10, 200, 50)
n_iter = st.sidebar.slider("İterasyon", 10, 500, 100)
decay = st.sidebar.slider("Decay", 0.0, 1.0, 0.95)
alpha = st.sidebar.slider("Alpha", 0.0, 5.0, 1.0)
beta = st.sidebar.slider("Beta", 0.0, 5.0, 2.0)

if st.sidebar.button("ROTAYI HESAPLA"):
    if not api_key:
        st.error("Lütfen önce API Anahtarını girin!")
        st.stop()

    try:
        gmaps = googlemaps.Client(key=api_key)
    except:
        st.error("API Anahtarı geçersiz.")
        st.stop()

    lats, valid_idx = [], []
    with st.spinner("Harita verileri alınıyor..."):
        for i, s in enumerate(SCHOOLS):
            lt, ln = get_coordinates(s, gmaps)
            if lt: lats.append((lt, ln)); valid_idx.append(i)

        if len(lats) < 2: st.error("Yeterli veri yok."); st.stop()
        matrix = create_distance_matrix([SCHOOLS[i] for i in valid_idx], gmaps)

    with st.spinner("Rota optimize ediliyor..."):
        aco = AntColonyOptimization(matrix, n_ants, int(n_ants/2), n_iter, decay, alpha, beta)
        path, dist, history = aco.run()

    data = {"dist": dist, "path": path, "history": history, "lats": lats, "valid_idx": valid_idx}
    with open(SAVE_FILE, "wb") as f: pickle.dump(data, f)

    st.success("Bitti!")
    time.sleep(0.5)
    st.rerun()

if os.path.exists(SAVE_FILE):
    try:
        with open(SAVE_FILE, "rb") as f: res = pickle.load(f)
        c1, c2 = st.columns([2, 1])
        with c1:
            st.info(f"✅ Mesafe: {int(res['dist'])} metre")
            m = folium.Map(location=[res['lats'][0][0], res['lats'][0][1]], zoom_start=12)

            folium.PolyLine([res['lats'][i] for i in res['path']], color="red", weight=4).add_to(m)

            for i, idx in enumerate(res['path']):
                folium.Marker(res['lats'][idx], tooltip=f"{i}. Durak").add_to(m)

            st_folium(m, width=700)

        with c2:
            st.write("### 📉 Yakınsama Grafiği")
            # GRAFİK DÜZELTİLDİ: Native Chart Kullanımı
            chart_data = pd.DataFrame(res['history'], columns=["Mesafe"])
            st.line_chart(chart_data)

            st.write("### 📋 Ziyaret Sırası")
            for i, idx in enumerate(res['path']):
                st.write(f"**{i}.** {SCHOOLS[res['valid_idx'][idx]]}")

        if st.sidebar.button("Temizle"):
            os.remove(SAVE_FILE)
            st.rerun()
    except Exception as e:
        st.error(f"Hata: {e}")
''')

# ---------------------------------------------------------
# 6. DOSYA: requirements.txt (PANDAS EKLENDİ)
# ---------------------------------------------------------
with open(f"{base_dir}/requirements.txt", "w") as f:
    f.write("streamlit\ngooglemaps\nnumpy\nmatplotlib\nfolium\nstreamlit-folium\npandas\n")

# ---------------------------------------------------------
# 7. DOSYA: secrets.toml (BOŞ ŞABLON)
# ---------------------------------------------------------
with open(f"{base_dir}/.streamlit/secrets.toml", "w") as f:
    f.write('google_maps_api_key = ""')

# ---------------------------------------------------------
# 8. DOSYA: README.md
# ---------------------------------------------------------
with open(f"{base_dir}/README.md", "w", encoding="utf-8") as f:
    f.write("# ACO Projesi\nAPI Key güvenliği için secrets kullanılmıştır.")

print("✅ TÜM DOSYALAR HATASIZ (GRAFİK DÜZELTMELİ) OLUŞTURULDU.")

✅ TÜM DOSYALAR HATASIZ (GRAFİK DÜZELTMELİ) OLUŞTURULDU.


In [ ]:
import os
import shutil
import time
import yaml
from pyngrok import ngrok, conf

# --- 1. AYARLAR ---
NGROK_TOKEN = "374qllMC8xibNDDy1wSWgjn38cB_65wWJd8terLJi3BXe3Ead"

print("🧹 Sistem temizleniyor...")
os.system("pkill ngrok")
os.system("pkill streamlit")

# --- 2. KONFİGÜRASYONU MANUEL ONAR ---
config_dir = "/root/.config/ngrok"
if os.path.exists(config_dir):
    shutil.rmtree(config_dir)
os.makedirs(config_dir, exist_ok=True)

# YAML dosyasını elle oluştur (En garantisi)
config_path = os.path.join(config_dir, "ngrok.yml")
data = {"authtoken": NGROK_TOKEN, "version": "2"}
with open(config_path, "w") as f:
    yaml.dump(data, f)

conf.get_default().config_path = config_path
print("✅ Ayarlar onarıldı.")

# --- 3. STREAMLIT'İ BAŞLAT ---
print("⏳ Streamlit başlatılıyor...")
os.chdir("/content/aco_ilac_rutasi")
# CORS ve XSRF'yi kapatıyoruz ki gri ekran sorunu olmasın
cmd = "nohup streamlit run main.py --server.port 8501 --server.enableCORS=false --server.enableXsrfProtection=false > streamlit.log 2>&1 &"
os.system(cmd)
time.sleep(5)

# --- 4. BAĞLANTIYI AÇ ---
try:
    url = ngrok.connect(8501).public_url
    print(f"\n🚀 --------------------------------------------------")
    print(f"🚀 UYGULAMANIZ HAZIR: {url}")
    print(f"🚀 --------------------------------------------------")
    print("⚠️ Not: Linke tıkladığında 'Visit Site' butonuna bas.")
    print("⚠️ Sayfa açılınca API KEY girmeyi unutma!")
except Exception as e:
    print(f"❌ Bağlantı hatası: {e}")

🧹 Sistem temizleniyor...
✅ Ayarlar onarıldı.
⏳ Streamlit başlatılıyor...

🚀 --------------------------------------------------
🚀 UYGULAMANIZ HAZIR: https://caulicolous-unviolently-alisha.ngrok-free.dev
🚀 --------------------------------------------------
⚠️ Not: Linke tıkladığında 'Visit Site' butonuna bas.
⚠️ Sayfa açılınca API KEY girmeyi unutma!


In [ ]:
import shutil
from google.colab import files

# 1. Klasörü ZIP haline getir
print("📦 Dosyalar paketleniyor...")
shutil.make_archive("/content/aco_proje_teslim", 'zip', "/content/aco_ilac_rutasi")

# 2. İndirmeyi başlat
print("⬇️ İndirme başlıyor...")
files.download("/content/aco_proje_teslim.zip")

📦 Dosyalar paketleniyor...
⬇️ İndirme başlıyor...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>